In [1]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
from LSTM_Attention import SimpleModel as Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv

from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import itertools
import argparse
import os,sys,re
import numpy as np 
from Bio import SeqIO
import pickle as pkl
import scipy.io as sio
%matplotlib inline

In [2]:
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

def Binary(sequences):
    AA = 'ACGU'
    binary_feature = []
    for seq in sequences:
        # seq=str(seq)[2:23]
        binary = []
        for aa in seq:
            binary_one = []
            for aa1 in AA:
                tag = 1 if aa == aa1 else 0
                binary_one.append(tag)
            #binary.append(binary_one)
            binary.append(binary_one)
        binary_feature.append(binary)
    return np.array(binary_feature)

train_seq = fa_seq("data/Mouse/mouse_train.fasta")
trainData = Binary(train_seq)
trainLabel = np.append(np.ones(int(len(trainData)/2)),np.zeros(int(len(trainData)-len(trainData)/2)),axis=0)

In [3]:
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

class EmbeddingSeq(nn.Module):
    def __init__(self,weight_dict_path):
        """
        Inputs:
            weight_dict_path: path of pre-trained embeddings of RNA/dictionary
        """
        super(EmbeddingSeq,self).__init__()
        weight_dict = pickle.load(open(weight_dict_path,'rb'))

        weights = torch.FloatTensor(list(weight_dict.values())).cuda()
        num_embeddings = len(list(weight_dict.keys()))
        embedding_dim = 300

        self.embedding = nn.Embedding(num_embeddings=num_embeddings,embedding_dim=embedding_dim)
        self.embedding.weight = nn.Parameter(weights)
        self.embedding.weight.requires_grad = False

    def forward(self,x):

        out = self.embedding(x.type(torch.cuda.LongTensor))

        return out

def seq2index(seqs,my_dict,window=3,save_data=False):
    """
    Convert single RNA sequences to k-mers representation.
        Inputs: ['ACAUG','CAACC',...] of equal length RNA seqs
        Example: 'ACAUG' ----> [ACA,CAU,AUG] ---->[21,34,31]
    """

    num_samples = len(seqs)
    temp = []
    for k in range(num_samples):
        length = len(seqs[k])
        seqs_kmers = [seqs[k][i:i+window] for i in range(0,length-window+1)]
        temp.append(seqs_kmers)


    seq_kmers = pd.DataFrame(data = np.concatenate(temp,axis=0))

    # load pretained word2vec embeddings

    word2index = word2index_(my_dict)

    seq_kmers_index = seq_kmers.applymap(lambda x: mapfun(x,my_dict))


    return seq_kmers_index.to_numpy()
def word2index_(my_dict):
    word2index = dict()
    for index, ele in enumerate(list(my_dict.keys())):
        word2index[ele] = index

    return word2index

def mapfun(x,my_dict):
    if x not in list(my_dict.keys()):
        return None
    else:
        return word2index_(my_dict)[x]
    
embed = EmbeddingSeq('features/embeddings_12RM.pkl')
embeddings_dict = pickle.load(open('features/embeddings_12RM.pkl','rb'))
train_seq = fa_seq("data/Mouse/mouse_train.fasta")
trainLabel = np.append(np.ones(int(len(train_seq)/2)),np.zeros(int(len(train_seq)-len(train_seq)/2)),axis=0)
seq_T = []
for i in train_seq:
    seq_T.append(i.replace('U','T'))
x = seq2index(seq_T,embeddings_dict)
seqs_kmers_index = torch.transpose(torch.from_numpy(x),0,1)
trainData = np.array(embed(seqs_kmers_index).view(len(train_seq),-1,300).cpu())

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [4]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

def attention(query, key, value, mask=None, dropout=None):  # q,k,v: [batch, h, seq_len, d_k]
    d_k = query.size(-1)  # query的维度
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)  # 打分机制 [batch, h, seq_len, seq_len]
    p_atten = F.softmax(scores, dim=-1)  # 对最后一个维度归一化得分, [batch, h, seq_len, seq_len]
    if dropout is not None:
        p_atten = dropout(p_atten)
    return torch.matmul(p_atten, value), p_atten  # [batch, h, seq_len, d_k] 作矩阵的乘法

class PositionalEncoding(nn.Module):

    def __init__(self, dim1, dim2, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()

        #if dim % 2 != 0:
        #    raise ValueError("Cannot use sin/cos positional encoding with "
        #                     "odd dim (got dim={:d})".format(dim))

        """
        构建位置编码pe
        pe公式为：
        PE(pos,2i/2i+1) = sin/cos(pos/10000^{2i/d_{model}})
        """
        pe = torch.zeros(max_len, dim2)  # max_len 是解码器生成句子的最长的长度，假设是 10
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term0 = torch.exp((torch.arange(0, dim2, 2, dtype=torch.float) * -(math.log(10000.0) / dim2)))
        div_term1 = torch.exp((torch.arange(1, dim2, 2, dtype=torch.float) * -(math.log(10000.0) / dim2)))
        
        pe[:, 0::2] = torch.sin(position.float() * div_term0)
        pe[:, 1::2] = torch.cos(position.float() * div_term1)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        #self.drop_out = nn.Dropout(p=dropout)
        self.dim2 = dim2
        self.bm1 = nn.BatchNorm1d(dim1,eps=1e-05)

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim2)
        if step is None:
            emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
        else:
            emb = emb + self.pe[step]
        #emb = self.drop_out(emb)
        emb = self.bm1(emb.to(torch.float32))
        return emb
    
def clones(module, N):  #定义clones方法
    return nn.ModuleList([copy.deepcopy(module)
                          for _ in range(N)])  #让原来变量不影响,且克隆module N次

class SelfAttention(nn.Module):  #多头注意力机制

    def __init__(self,embedding_dim, dropout=0.1):
        super(SelfAttention, self).__init__()
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)  #克隆四份Linear网络层
        self.dropout = nn.Dropout(p=dropout)  #定义Dropout层

    def forward(self,query,key,value,mask=None):  # q,k,v: [batch, seq_len, embedding_dim]
        nbatches = query.shape[0]  #批数量
        query, key, value = [
            l(x) for l, x in zip(self.linears,
                            (query.to(torch.float32),
                             key.to(torch.float32),
                             value.to(torch.float32)))
        ]  #获取zip的query,key,value权重矩阵
        attn, p_atten = attention(query,key,value,mask=mask,dropout=self.dropout)
        out = self.linears[-1](attn)  #得到最后一层线性层的输出
        return out,p_atten  #返回out结果

In [8]:
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import copy
from torch.autograd import Variable
from sklearn.model_selection import KFold

def train(model,data,label,epoch,train_device,model_dir,batch_size):
    if os.path.exists(model_dir+'model.pt'):
        model_train = torch.load(model_dir+'/model.pt')
    else:
        model_train = model
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model_train.parameters(),lr=0.003)  #改变学习率
    dataX = torch.Tensor(data).clone().detach()
    label = torch.Tensor(label).clone().detach()
    train_data = TensorDataset(dataX, label)
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    running_loss = 0.0
    model_train = model_train.to(train_device)
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        #inputs = inputs.reshape(inputs.shape[0], 1, inputs.shape[1])
        inputs = inputs.to(train_device)
        target = target.to(train_device)
        target = target.reshape(target.shape[0],1)
        optimizer.zero_grad()
        outputs = model_train(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        #print(type(loss))
        running_loss += loss.item()
        if batch_idx == len(dataX)//batch_size:
            #print('[%d, %5d] epoch loss: %.3f' %(epoch+1,batch_idx+1,running_loss))
            print(running_loss)
    save_model(model_train,model_dir)
    model_train = torch.load(model_dir+'/model.pt')
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    th,_,_,_,_,_,_,_,_,_ = metricsCal.evaluate(model_train,train_loader,train_device)
    return running_loss,th

def test(data,label,best_auc,test_device,model_dir,batch_size,th):
    model_test = load_model(model_dir)
    #model_test.eval()
    #model_test.to(test_device)
    data = torch.Tensor(data).clone().detach()#torch.Tensor(data)
    label = torch.Tensor(label).clone().detach()#.requires_grad_(True)torch.Tensor(label)
    test_data = TensorDataset(data,label)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    _,_,_,_,Sen,Spe, Acc, mcc, AUC = metricsCal.evaluate(model_test,test_loader,test_device,False,th)
    
    print('Accuracy on test set: %d %%' %Acc)
    print('Sensitivity on test set: %d %%' %Sen)
    print('Speciality on test set: %d %%' %Spe)
    print('MCC on test set: %.3f' %mcc)
    print('auc on test set: %.3f' %AUC)
    if(AUC > best_auc):
        torch.save(model_test,model_dir+'model_best.pt')
    return Sen,Spe,Acc, mcc, AUC

def independTest(data,label,test_device,model_dir,batch_size,th):
    model_test = load_bestModel(model_dir)
    data = torch.Tensor(data).clone().detach()#torch.Tensor(data)
    label = torch.Tensor(label).clone().detach()#.requires_grad_(True)torch.Tensor(label)
    test_data = TensorDataset(data,label)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    _,_,_,_,Sen,Spe, Acc, mcc, AUC = metricsCal.evaluate(model_test,test_loader,test_device,False,th)
    print(Acc,mcc,AUC)
    print('Accuracy on test set: %d %%' %Acc)
    print('Sensitivity on test set: %d %%' %Sen)
    print('Speciality on test set: %d %%' %Spe)
    print('MCC on test set: %.3f' %mcc)
    print('auc on test set: %.3f' %AUC)
    return Acc, mcc, AUC

def load_model(model_dir):
    if os.path.exists(model_dir+'model.pt'):
        model_load = torch.load(model_dir+'model.pt')
    return model_load

def save_model(model_save,model_dir):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    torch.save(model_save, model_dir+'model.pt')

def load_bestModel(model_dir):
    if os.path.exists(model_dir+'model_best.pt'):
        model_load = torch.load(model_dir+'model_best.pt')#,map_location='cuda:0')
    return model_load

In [6]:
#CNN
class Model_CNN(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1,3,kernel_size=5,stride=2,padding=2)
        self.conv2 = nn.Conv2d(3,3,kernel_size=5,stride=2,padding=2)
        self.conv3 = nn.Conv2d(3,3,kernel_size=5,stride=2,padding=2)
        
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        k1 = int((k1+2*2-5)/2)+1
        k2 = int((k2+2*2-5)/2)+1
        k1 = int((k1+2*2-5)/2)+1
        k2 = int((k2+2*2-5)/2)+1
        
        self.fn1 = nn.Linear(3*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        
        x1 = x.contiguous().view(-1,1,x.shape[1],x.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = self.conv2(x1)
        x1 = self.relu(x1)
        x1 = self.conv3(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

#LSTM
class Model_LSTM(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_LSTM, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=False)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        #self.fn1 = nn.Linear(dim1*dim2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = self.relu(x1)
        x1 = self.bm1(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

#BiLSTM
class Model_BiLSTM(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_BiLSTM, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=True)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = x1[:,:,0:300]+x1[:,:,300:600]
        x1 = self.relu(x1)
        x1 = self.bm1(x1)
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out
    
#Attention
class Model_Attention(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_Attention, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.self_A = SelfAttention(dim2)
        self.self_B = SelfAttention(dim1)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        if x1.shape[1] == 1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
            x1,attn = self.self_B(x1,x1,x1)
        else:
            x1,attn = self.self_A(x1,x1,x1)
        if x1.shape[2]==1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
        x1 = self.bm1(x1)
        x1 = self.relu(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

#LSTM_Attention_CNN
class Model_LSTM_Attention_CNN(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_LSTM_Attention_CNN, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.self_A = SelfAttention(dim2)
        self.self_B = SelfAttention(dim1)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=False)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)
        #self.encoder = nn.Sequential()

    def forward(self, x):
        
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = self.relu(x1)
        if x1.shape[1] == 1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
            x1,attn = self.self_B(x1,x1,x1)
        else:
            x1,attn = self.self_A(x1,x1,x1)
        if x1.shape[2]==1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
        x1 = self.bm1(x1)
        x1 = self.relu(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out
    
#BiLSTM_Attention_CNN
class Model_BiLSTM_Attention_CNN(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_BiLSTM_Attention_CNN, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.self_A = SelfAttention(dim2)
        self.self_B = SelfAttention(dim1)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=True)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = x1[:,:,0:300]+x1[:,:,300:600]
        x1 = self.relu(x1)
        if x1.shape[1] == 1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
            x1,attn = self.self_B(x1,x1,x1)
        else:
            x1,attn = self.self_A(x1,x1,x1)
        if x1.shape[2]==1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
        x1 = self.bm1(x1)
        x1 = self.relu(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

In [9]:
#先做交叉验证，看看多少个epoch合适
rows = 10
row = -1
max_epochs = 150
max_patience = 25
batch_size = 256
row_name = []
Loss = np.zeros(rows*max_epochs).reshape(rows,max_epochs)
Sen = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Spe = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Acc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Mcc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Auc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
th_All = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))

#然后全部all_in 一个模型出来
#%env CUDA_LAUNCH_BLOCKING=1
kf = KFold(10,True,0)
#第三重循环
for i,[train_index, test_index] in enumerate(kf.split(trainData)):
    row+=1
    row_one = "LSTM_Attention_CNN_w2v_KFold_" + str(i)
    row_name.append(row_one)
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    best_auc = 0
    patience = 0
    model_dir = "Model/Model_Trans_CNN_RuLU_w2v/KFold_" + str(i)+"/"
    model = TransCNN()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
    model = model.to(device)
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'

    for j in range(max_epochs):
        runningLoss,th = train(model,X_train,Y_train,j,device,model_dir,batch_size)
        sen,spe, acc, mcc, auc= test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
        Loss[row][j] = runningLoss
        th_All[row][j] = th
        Sen[row][j] = sen
        Spe[row][j] = spe
        Acc[row][j] = acc
        Mcc[row][j] = mcc
        Auc[row][j] = auc                            
        if auc > best_auc:
            best_auc=auc
            patience=0
        else:
            patience+=1
            
        if patience>max_patience and best_auc!= 0.5:
            break

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


23.844664692878723
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.115
auc on test set: 0.595
23.986645698547363
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.118
auc on test set: 0.587
23.188163340091705
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.207
auc on test set: 0.636
23.086291253566742
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.068
auc on test set: 0.531
23.013870298862457
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.055
auc on test set: 0.477
23.026685774326324
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: -0.006
auc on test set: 0.496
23.008689045906067
Accuracy on test set: 0 %
Sensitivity on test set: 1 %
Speciality on test set: 0 %
MCC on te

22.907622277736664
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.070
auc on test set: 0.511
22.92661440372467
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.195
auc on test set: 0.642
22.912088334560394
Accuracy on test set: 0 %
Sensitivity on test set: 1 %
Speciality on test set: 0 %
MCC on test set: 0.000
auc on test set: 0.500
22.956439435482025
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.178
auc on test set: 0.588
22.920087695121765
Accuracy on test set: 0 %
Sensitivity on test set: 1 %
Speciality on test set: 0 %
MCC on test set: 0.000
auc on test set: 0.500


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [11]:
np.max(Auc,axis=1)

array([0.63551774, 0.71199597, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [14]:
np.max(pd.read_csv("Model/w2v_BiLSTM_Attention_CNN.csv").iloc[-10:,2:],axis=1)

0.8048860375000648

In [51]:
Row_name = []
name = ["Loss","Thr","Sen","Spe","Acc","Mcc","Auc"]
for i in range(7):
    for j in range(10):
        Row_name.append(name[i]+"_KFold_"+str(j))

In [124]:
pd.DataFrame(np.hstack((np.array(Row_name).reshape(-1,1),np.vstack((Loss,th_All[:,:150],Sen[:,:150],Spe[:,:150],Acc[:,:150],Mcc[:,:150],Auc[:,:150]))))).to_csv("Model/w2v_BiLSTM.csv")#,header=["Name","Loss","Thr","Sen","Spe","Acc","Mcc","Auc"])

In [ ]:
#LSTM+Attn+CNN  0.811
# array([0.8006344 , 0.81798675, 0.82836602, 0.80211214, 0.81431039,
#        0.81716833, 0.80938502, 0.79952567, 0.82470794, 0.80080841])

#CNN
# array([0.78121371, 0.80199213, 0.80797362, 0.79136905, 0.81091276,
#        0.81586777, 0.81027476, 0.79999711, 0.81386285, 0.79199288])

In [7]:
from typing import Optional
from torch import Tensor
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, max_len: int = 416):#将64改为416
        super().__init__()

        position = torch.arange(max_len).unsqueeze(1).unsqueeze(2) # [maxlen, 1, 1]
        div_term = 2 * math.pi * torch.exp(torch.arange(0, d_model, 2) * (-math.log(512.0) / d_model))
        div_term = div_term.unsqueeze(0).unsqueeze(0) # [1, 1,  d_model // 2]
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, :, 0::2] = torch.sin(position * div_term)
        pe[:, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0), :, :]
        return x
class TransformerEncoderLayer(nn.Module):

    def __init__(self, d_model, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, 1, dropout=dropout)
        
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = nn.ReLU()

    def with_pos_embed(self, tensor, pos: Optional[Tensor]):
        return tensor if pos is None else tensor + pos

    def forward(self,
                     src,
                     src_mask: Optional[Tensor] = None,
                     src_key_padding_mask: Optional[Tensor] = None,
                     pos: Optional[Tensor] = None):
        q = k = self.with_pos_embed(src, pos)
        src2 = self.self_attn(q, k, value=src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))#######MLP############
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        
        return src

class TransCNN(nn.Module):
    def __init__(self, input_dim=39, ouput_dim=64, dropout=0.1):
        super(TransCNN, self).__init__()
        self.ouput_dim = ouput_dim

        self.conv1 = nn.Conv1d(input_dim, ouput_dim, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(ouput_dim, ouput_dim, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(ouput_dim, ouput_dim, kernel_size=3, stride=2, padding=1)
        
        self.pos_embed = PositionalEncoding(ouput_dim)
        self.query_embed = nn.Embedding(1, ouput_dim)

        self.encoders = nn.ModuleList([
            TransformerEncoderLayer(ouput_dim, dim_feedforward=4*ouput_dim, dropout=dropout)
            for _ in range(3)
        ])

        self.linear_1 = nn.Linear(64, 4 * ouput_dim)
        self.linear_2 = nn.Linear(4 * ouput_dim, ouput_dim)
        self.linear_3 = nn.Linear(ouput_dim, 1)
        self.relu = nn.ReLU()
        self.predictor = nn.Sequential(self.linear_1, self.relu, self.linear_2, self.relu, self.linear_3)

    def forward(self, x):
        
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        
        x = x.permute(2, 0, 1) # [B, C, T] -> [T, B, C]
        

        query_embed = self.query_embed.weight.unsqueeze(1).repeat(1, x.shape[1], 1) # [1, B, C]
        #print(query_embed.shape)
        x = torch.cat([query_embed, x], dim=0) # [T + 1, B, C]
        
        pos = self.pos_embed(x)
        for encoder in self.encoders:
            x = encoder(x, pos=pos)
        out = self.predictor(x[0])

        return out

In [16]:
net = TransCNN()

In [17]:
net(torch.Tensor(trainData[0:100])).shape

torch.Size([100, 1])

In [18]:
import torch

# 选择要使用的设备，如cuda:0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 创建一个Tensor对象并将其移动到设备上
tensor = torch.Tensor([0]).to(device)

# 将Tensor对象分配给另一个新的变量以发起销毁操作
del tensor